显然大一些的模型 , 单独配置 config 文件是必要的

一些参量的含义 : 
- seq_len	序列的最大长度
- datasource	数据集名称
- lang_src / lang_tgt	源语言 / 目标语言
- model_folder	模型权重保存的目录 ( 这个是文件夹名 )
- model_basename	模型文件名前缀 ( 用于多 epoch 保存 ) ( 这个是文件名 )
- preload	是否加载最新的模型 (latest) ( 在 train.py 中 , 如果为真则会调用 )
- tokenizer_file	tokenizer 文件名模板
- experiment_name	用于 TensorBoard 记录等实验标识路径

tokenizer_file 还要分文件存 , 是因为其是面向不同语言的 , 每一个语言存一份 . 

而其名字里的 `{0}` 是搭配将来的 format 格式化用的 , python 允许提前用其来占位 , 使用时直接 `xxx.format()` 就行了 ( 比如 : `tokenizer_src_path = config["tokenizer_file"].format(config["lang_src"])` ) . 

下面又提供了两个调用模型参数的函数 : 

注意到它们的主体是相同的 , 用来返回某epoch下的参数来防止每次从零开始 , 不同的是一个可以手动传从第几个 epoch 开始 , 一个自动找最新的 epoch , 

也就与 config 里的 latest 参数相呼应了 . 

In [ ]:
from pathlib import Path

def get_config():
    return {
        "batch_size": 4,
        "num_epochs": 20,
        "lr": 10**-4,
        "seq_len": 350,
        "d_model": 512,
        "datasource": 'opus_books',
        "lang_src": "en",
        "lang_tgt": "it",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": "latest",
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

def get_weights_file_path(config, epoch: str):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)

# Find the latest weights file in the weights folder
def latest_weights_file_path(config):
    model_folder = f"{config['datasource']}_{config['model_folder']}"
    model_filename = f"{config['model_basename']}*"
    weights_files = list(Path(model_folder).glob(model_filename))
    if len(weights_files) == 0:
        return None
    weights_files.sort()
    return str(weights_files[-1])